In [2]:
# comment out to run on drive + colab

!pip install transformers_interpret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 898.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [4]:
# from google.colab import drive
#

drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.1 MB/s eta 0:00:00


In [6]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers_interpret import MultiLabelClassificationExplainer
import torch
from transformers import pipeline
from datasets import load_from_disk

import warnings
warnings.filterwarnings('ignore')

In [7]:
# comment out the filepath to run on drive + colab
filepath = "../models/scenario_level/anomic_control"
# filepath = "/content/drive/Shareddrives/AphasiaProject/models/scenario_level/anomic_control"

model = AutoModelForSequenceClassification.from_pretrained(filepath+"/model")
tokenizer = AutoTokenizer.from_pretrained(filepath+"/tokenizer", padding=True,
                                          truncation=True, return_tensors="pt",
                                          add_special_tokens=True, max_length=512)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == torch.device("cuda"):
    model.to(device)
pipe = pipeline("text-classification", model=filepath+"/model", tokenizer=filepath+"/tokenizer", device=device, truncation=True, padding=True)
cls_explainer = MultiLabelClassificationExplainer(model, tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
model.config

DistilBertConfig {
  "_name_or_path": "/content/drive/Shareddrives/AphasiaProject/models/anomic_control/model",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "CONTROL",
    "1": "ANOMIC"
  },
  "initializer_range": 0.02,
  "label2id": {
    "ANOMIC": 1,
    "CONTROL": 0
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.38.2",
  "vocab_size": 30524
}

In [9]:
model.config.id2label

{0: 'CONTROL', 1: 'ANOMIC'}

In [10]:
def interpret(data):
    for text, label in zip(list(data["new_preprocessed_text"]), list(data["label"])):
        if device == torch.device("cuda"):
            inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt", add_special_tokens=True, max_length=512).input_ids.to(device)
            # tranformers interpret gives error for input of 512 tokens
            if len(inputs[0]) != 512:
                with torch.no_grad():
                    logits = model(inputs).logits

                predicted_class_id = logits.argmax().item()
                print("True: ", label, "Pred: ", model.config.id2label[predicted_class_id])
                print("Text: ", text)
                cls_explainer(text)
                cls_explainer.visualize()
                print("----------------------------------------------------------------------")
        else:
            inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt", add_special_tokens=True, max_length=512)
            # tranformers interpret gives error for input of 512 tokens
            if len(inputs[0]) != 512:
                with torch.no_grad():
                    logits = model(**inputs).logits

                predicted_class_id = logits.argmax().item()
                print("True: ", label, "Pred: ", model.config.id2label[predicted_class_id])
                print("Text: ", text)
                cls_explainer(text)
                cls_explainer.visualize()
                print("----------------------------------------------------------------------")

In [11]:
scenarios = ["Speech", "Important_Event", "Cinderella", "Stroke", "Cat"]
# make sure not to interpret on trained scenarios
texts = load_from_disk(filepath + "/dataset")["test"].remove_columns(["Unnamed: 0", "input_ids", "attention_mask"]).to_pandas()
texts["label"] = [model.config.id2label[x] for x in texts["label"]]
n = 3

# Speech scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [12]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[0])].sample(n=n)])

In [13]:
interpret(data)

True:  ANOMIC Pred:  ANOMIC
Text:  yeah. i can remember more words. but not as good as i should. oh yeah.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.12),CONTROL,-3.77,[CLS] yeah . i can remember more words . but not as good as i should . oh yeah . [SEP]
n/a,(0.86),ANOMIC,3.87,[CLS] yeah . i can remember more words . but not as good as i should . oh yeah . [SEP]


----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  okay. i [FP] [FP] [FP] i speech [FP] [FP] is pretty good. on on in the in the morning it's pretty good. on the afternoon and the evening it loses it. yeah yeah. that's all i'm doing. but somebody said. well you know you're you're you're you're tired you know. you can't stop stop. no i can but i can't you know. yeah.


----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  [FP] [FP] [FP] [FP] [FP] slowly recovering. [FP] i go to the bar and [FP] and fast eddie's bar and [FP] play [FP] video games of poker. and [FP] once in awhile i i have a beer and [FP] tea. [FP] i'll drink tea. and [FP] [FP] [FP] [FP] firstnames my wife [FP] is a delta employee [FP] no [FP] retired and [FP] [FP] firstnamed lastname [FP] well a lobbyist [FP] a secretary for a lobbyist [FP] in downtown dc.


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  but [FP] at my age i very often [FP] forget the word that i want to say. and it's getting a little worse. names especially.


----------------------------------------------------------------------
True:  CONTROL Pred:  ANOMIC
Text:  spanish.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.44),CONTROL,-0.33,[CLS] spanish . [SEP]
n/a,(0.55),ANOMIC,0.44,[CLS] spanish . [SEP]


----------------------------------------------------------------------
True:  CONTROL Pred:  ANOMIC
Text:  no i do not.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.13),CONTROL,-2.11,[CLS] no i do not . [SEP]
n/a,(0.85),ANOMIC,2.10,[CLS] no i do not . [SEP]


----------------------------------------------------------------------


# Important event scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [14]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[1])].sample(n=n)])

In [15]:
interpret(data)

True:  ANOMIC Pred:  ANOMIC
Text:  okay. [FP] went to school at jamaica. [FP] head boy. so [FP] head boy and wear epaulets and a tie. no i can't. i don't remember. yeah okay.


----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  my i they i has chest pain. and i said. [FP] it must be caused. but i i said. i it isn't cold [FP] it isn't cold. i i was walking in the park every day. he said. i i did. i was i i that was the only thing that i that i really wanted. that want wanted. it's what and we were. that was sometime sometime in the fall. and then i i was i was glad to get it over with. no. no he said i they couldn't say anything. yeah right. yup.


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  [FP] sad memory when my father passed away. we've been married for twenty four years. [FP] very happily married. and we are living in cityname maryland. three kids. living the dream!


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  and the people in georgetown said the place was haunted. i did not believe it. and there would people would come in from out of town and say. there are voices upstairs. can you hear? and i would go upstairs. i could hear nothing. and still other people some people would come in. and soon as they step in the building they say. this place is eerie. it's eerie. i'm leaving. and they would turn around and leave. fact one lady stepped about three or four steps in the door. she started screaming. this place is eerie. she grabbed her purse. and started running down the street. i still never heard anything. until one day i was there by myself. and we had a lace curtain on the front door. all of a sudden the lace curtain started going up raising up and up and ascending. went up about about four feet high. and about three feet from the door. and it went up very very slowly. and then when i

----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  [FP] [FP] means we'll go with our marriage. [FP] we didn't want to have a church ceremony. so we found the jp. this is in san diego california by the way. [FP] we found the jp but he was way the heck and gone east of san diego. so we drove out to this little teensy tiny town called boulevard and had a ceremony with the jp and then drove all the way back. and then a few months later we learned that there was actually another jp in town about two miles away. so but it was still interesting. not too many people know where boulevard is in san diego.


----------------------------------------------------------------------


# Cinderella scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [16]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[2])].sample(n=n)])

In [ ]:
interpret(data)

True:  ANOMIC Pred:  ANOMIC
Text:  [FP] [FP] beautiful to everything. and so [FP] this little aunt mary somebody comes by and and says. oh i can help you with with something then. i can [FP] dress you all up and and. then she said. okay alright. so she [FP] gets her all fix her up and all that. and [FP] there then she [FP] gets her back life wife with where she's gonna be like a servant school or wife. because she's a in a shop. and here comes the evil looking looking mother. and so as she as she's doing her [FP] gown and good and her her well with her. no there's just setting the the table i guess. and she's [FP] doing all this kitchen territories and all that stuff. and [FP] it's [FP] very sad. but the window the [FP] the fairy cames in another day and gives her boom alright. [FP] a lovely dress for her to go to the ball that the other sisters and the sisters for her hafta go. and now she's into one and boom. there the one wonderful man man feel bad for her. so [FP] they [FP] [FP] fa

----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  and the animals helped cinderella [FP] [FP] [FP] helped cinderella with her dress. cinderella and [FP] and was swimming [FP] [FP] dancing with the prince. and then [FP] she had said that they hadta [FP] she hadta [FP] leave before midnight. and the ugly stepmother they locked her into locked her away. and in the end the the mice got her key. and the the people from the kingdom hadta [FP] try on the [FP] the shoe. and the [FP] prince and the prince said. well anyway the shoe was [FP] felt fit. not felt. [FP] the store the [FP] shoe [FP] felt. it's not the word felt. what're you saying on your shoe? fits. the shoe didn't it should fit cinderella. and the the [FP] the [FP] prince and the and cinderella lived happily after that.


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  [FP] cinderella is also very excited to meet her new sisters and her new stepmother. [FP] but when they do move in they treat cinderella very poorly. they make her clean. and they force her to live outside in the barn. and cinderella is very saddened by this. [FP] while she's cleaning she overhears her stepmother [FP] reading an invitation to her sisters that there's going to be a big royal ball at the castle that weekend. [FP] cinderella is very excited. while she is cleaning the attic [FP] she notices a book of spells in a treasure chest. and she uses that book to wish for a dress. the dress appears. and she puts it on and likes it. [FP] that weekend she she's prepares for the royal ball. and she puts on the dress and gets ready to leave with her sisters and stepmother. when her sisters and stepmother see that she is [FP] ready to go to the royal ball they rip off her jewelry a

----------------------------------------------------------------------


# Stroke scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [ ]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    df = texts[(texts["label"] == x) & (texts["scenario"] == scenarios[3])]
    # no controls in stroke scenario
    if len(df) >= n:
        data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[3])].sample(n=n)])

In [ ]:
interpret(data)

True:  ANOMIC Pred:  ANOMIC
Text:  yes. that i had a stroke? i had a stroke in [FP] november about three years ago. here's one problem. last june thirtieth. a year ago. two thousand twelve. it was a really hot day. and and the electric cut out. and i was working the yard and drink enough water. i had four seizures. and i didn't know it. i went back to the hospital. i was out six days. and when i got up i was at some other building. i was there three weeks. again it was difficult to remember my words. some lady sit next to me [UP] for breakfast. and she says. hi how are you? and i go. oh shoot. i couldn't talk. so that was very difficult. i couldn't talk with her. and i was figure. oh shoot. what happened to me? that's when i started a problem. she came over for a week after my stroke. and she helped me. i have no idea she did that. i had another friend came over for two weeks to help me. no idea. so that whole time after my stroke it's all gone to remember it. going back for some years

----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  [FP] avm then stroke. doctor's hospital in san francisco. and doctor [FP] doctors. stroke in hospital. seventy three days [FP] got a call and that's it. [FP] better in [FP] auditory. not better in the chair. [FP] after the hospital good things. i like it.


----------------------------------------------------------------------


# Cat scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [ ]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[4])].sample(n=n)])

In [ ]:
interpret(data)

True:  ANOMIC Pred:  ANOMIC
Text:  a girl [FP] [FP] looks at cat. and a fire engine is coming. and the dog is barking. and a man no catch cat because [FP] i think [FP] because when he get there it breaks and he falls down. so firemen come with a ladder [FP] save him.


----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  come here come here come on. and then the dog is barking. and then [FP] the firemen has a ladder. and [FP] the cat was safe saved. the end. wow.


----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  [FP] the dog said. [FP] would you get the cat out of the tree? [FP] [FP] the dog said. woof woof. and the [UP] latter went up. and he's falled down. and the firetrucks come and get the ladder up to the tree. and she gets the cat out of the tree. and [FP] he was [FP] alright.


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  [FP] a cat went up a tree. and [FP] i guess a man tried to go up to get the cat. a dog is barking on the tree and probably knocked over the [FP] ladder. so the man couldn't get back down. so the fire department is coming to the rescue.


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  janie and her daddy are out playing in the park in the afternoon. and all of the sudden she looks up and sees her favorite kitty cat sitting on a limb in the tree. well she gets really upset. and she says. daddy you gotta do something about that. so daddy climbs the tree. and he gets up there and he gets stuck. and now we've got daddy and the cat up the tree. the dog comes along. sees them and thinks this is crazy. starts barking. so they call the fire department. and away comes the firemen. he's gonna they're gonna get the cat out of the tree. then they'll get dad out of the tree.


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  and there's a cat stuck in the tree. and there's a man stuck in the tree. and the little girl is in distress. and [FP] it looks like the the man was trying to rescue the cat but he got stuck too. and now the firemen are coming.


----------------------------------------------------------------------
